In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [3]:
import pandas as pd
from utils import helper, config, rayer
from ml.models.ensemble_v2 import Ensemble

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
rayer.get_global_cluster(num_cpus=5)
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-23 04:51:47.165723 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.909 GiB object_store_memory

Demands:
 {'CPU': 1}: 5+ from request_resources()


In [5]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-14 21:37:00.832013 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.908 GiB object_store_memory

Demands:
 {}: 1+ pending tasks/actors
 {'CPU': 1}: 5+ from request_resources()


In [6]:
# ray.autoscaler.sdk.request_resources(num_cpus=0)

In [8]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)

In [9]:
ss = StandardScaler()

X_train_scalar = pd.DataFrame(ss.fit_transform(X_train.copy()), columns=X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test.copy()), columns=X_test.columns)


In [22]:
list_base_models = ['BriskXGBoost', 'SlugXGBoost', 'SlugANN', 'SlugRF', 'SlugKNN', 'BriskBagging']
# list_base_models = ['BriskXGBoost', 'SlugXGBoost', 'BriskBagging', ]

ensemble_set = Ensemble(
                        list_base_models = list_base_models,
                        n_trials=500, ### for all models
                        boosted_round=100, ### for tree models only
                        epochs=150, ### for DNN models

                        ensemble_boosted_round=100, ### for ensemble model which is also a free model
                        ensemble_n_trials=100,
                                )


In [23]:
ensemble_set.fetch_models(X_train_scalar, X_test_scalar, y_train, y_test, threshold=None)

2022-11-23T05:00:44CST : INFO : ensemble_v2 : fetch_models : 108 : Message : Ensemble: starting discovery process for models [<ml.models.base.v2.brisk_xgboost.BriskXGBoost object at 0x7f25f8142640>, <ml.models.base.v2.slug_xgboost.SlugXGBoost object at 0x7f25f8142af0>, <ml.models.base.v2.slug_ann.SlugANN object at 0x7f25f8142460>, <ml.models.base.v2.slug_rf.SlugRF object at 0x7f25f81424f0>, <ml.models.base.v2.slug_knn.SlugKNN object at 0x7f25f81423d0>, <ml.models.base.v2.brisk_bagging.BriskBagging object at 0x7f25f81425b0>]
(__run_discoveries__ pid=3216) 2022-11-23T03:00:44PST : INFO : brisk_xgboost : __discover_model__ : 109 : Message : brisk_xgb: Starting training for trials:500, boosted rounds: 100, max depth: 10
(__run_discoveries__ pid=3216) /tmp/ray/session_2022-11-23_00-55-48_777969_8/runtime_resources/pip/6359c2ba15c7e71f0e73b7159cb8c36699f062e1/virtualenv/lib/python3.9/site-packages/optuna/_optimize.py:93: UserWarning: The default storage cannot be shared by multiple processes

(scheduler +8m56s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +8m56s) Adding 1 node(s) of type small-group.


(__run_discoveries__ pid=200, ip=10.10.105.191) 2022-11-23T03:00:51PST : INFO : slug_knn : __discover_model__ : 108 : Message : slug_knn: Number of trials: 500
(__run_discoveries__ pid=200, ip=10.10.105.191) 2022-11-23T03:00:51PST : INFO : slug_knn : __discover_model__ : 110 : Message : Best trial:13
(__run_discoveries__ pid=200, ip=10.10.105.191) 2022-11-23T03:00:51PST : INFO : slug_knn : __discover_model__ : 112 : Message :   Params: 
(__run_discoveries__ pid=200, ip=10.10.105.191) 2022-11-23T03:00:51PST : INFO : slug_knn : __discover_model__ : 114 : Message :     n_neighbors 6
(__run_discoveries__ pid=200, ip=10.10.105.191) 2022-11-23T03:00:51PST : INFO : slug_knn : __discover_model__ : 114 : Message :     weights uniform
(__run_discoveries__ pid=200, ip=10.10.105.191) 2022-11-23T03:00:51PST : INFO : slug_knn : __discover_model__ : 114 : Message :     algorithm kd_tree
(__run_discoveries__ pid=200, ip=10.10.105.191) 2022-11-23T03:00:51PST : INFO : slug_knn : __discover_model__ : 129

(scheduler +9m2s) Resized to 7 CPUs.


(__run_discoveries__ pid=238, ip=10.10.48.229) 2022-11-23T03:01:16PST : INFO : slug_xgboost : __discover_model__ : 120 : Message : slug_xgb: Number of trials: 500
(__run_discoveries__ pid=238, ip=10.10.48.229) 2022-11-23T03:01:16PST : INFO : slug_xgboost : __discover_model__ : 122 : Message : Best trial:192
(__run_discoveries__ pid=238, ip=10.10.48.229) 2022-11-23T03:01:16PST : INFO : slug_xgboost : __discover_model__ : 124 : Message :   Params: 
(__run_discoveries__ pid=238, ip=10.10.48.229) 2022-11-23T03:01:16PST : INFO : slug_xgboost : __discover_model__ : 126 : Message :     lambda 0.01550616475389052
(__run_discoveries__ pid=238, ip=10.10.48.229) 2022-11-23T03:01:16PST : INFO : slug_xgboost : __discover_model__ : 126 : Message :     alpha 0.05142429908610175
(__run_discoveries__ pid=238, ip=10.10.48.229) 2022-11-23T03:01:16PST : INFO : slug_xgboost : __discover_model__ : 126 : Message :     eta 0.784170380411157
(__run_discoveries__ pid=238, ip=10.10.48.229) 2022-11-23T03:01:16PST

In [27]:
ensemble_set.base_model_scores

[[0.9064236182823546,
  0.2971445245473846,
 [0.942306673581218,
  0.24762963825305173,
 [0.7887909740843149,
  0.3081144238034367,
 [0.8374187159048148,
  0.5055152119518554,
 [0.6497272678062903,
  0.5382569657433527,
 [0.9402230243935257,
  0.5962507086117166,

In [17]:
# ensemble_set.fit(X_train_scalar, X_test_scalar, y_train, y_test)

In [28]:
ensemble_set.score

[0.9919763222678744, 0.5957023193695732]

In [29]:
pred = ensemble_set.predict(X_test_scalar)
r2_score(pred, y_test)

-0.16032659316215825